<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M4_Basic_LSTM_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM in Pytorch

In [ ]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Basic LSTM in Pytorch with random numbers

In [ ]:
#input
x = torch.tensor([[1,2, 12,34, 56,78, 90,80],
                 [12,45, 99,67, 6,23, 77,82],
                 [3,24, 6,99, 12,56, 21,22]])

#### using two different models

In [ ]:
model1 = nn.Embedding(100, 7, padding_idx=0)
model2 = nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True)

In [ ]:
out1 = model1(x)
out2 = model2(out1)

In [ ]:
print(out1.shape)
print(out1)

torch.Size([3, 8, 7])
tensor([[[ 1.0279, -0.5072,  0.2478, -0.6202,  2.0033,  0.1640, -0.8012],
         [-0.3136, -0.0548,  1.5912,  1.4868,  0.2996, -0.7500,  0.0061],
         [ 0.6683,  0.8847, -0.4034,  0.9458, -0.2565, -0.6134, -2.3287],
         [ 0.2171, -1.4145, -0.7785,  0.8197, -1.6740,  0.6843,  0.8147],
         [-0.8618,  0.1090, -2.0859,  0.3951,  0.4770,  1.7297, -1.3735],
         [-0.3795, -0.3729, -1.1771,  0.8077, -0.0887,  0.8837,  2.1783],
         [ 0.3405, -0.7906,  0.2363, -0.6487, -1.0164,  1.0171, -0.9363],
         [ 1.3917,  0.6239,  0.7692,  1.9831,  0.3465,  0.5145,  1.4281]],

        [[ 0.6683,  0.8847, -0.4034,  0.9458, -0.2565, -0.6134, -2.3287],
         [-0.0522, -1.6644, -0.3441, -1.0254,  0.6189, -0.0996,  0.4633],
         [ 0.3129,  1.9830,  0.6658,  1.2399,  0.2724,  0.1949,  0.1003],
         [ 0.7731, -0.3001, -0.8024,  1.3366, -0.4056, -0.6406, -1.4317],
         [ 1.3162, -1.0460, -2.1511,  1.2025, -0.6545,  0.1758,  1.3846],
         [-0.9

In [ ]:
out, (ht, ct) = model2(out1)
print(ht)

tensor([[[-0.2904,  0.1176,  0.0496],
         [-0.0468, -0.0074,  0.2192],
         [-0.1647,  0.1382, -0.1190]]], grad_fn=<StackBackward0>)


#### using nn.sequential

In [ ]:
model3 = nn.Sequential(nn.Embedding(100, 7, padding_idx=0),
                        nn.LSTM(input_size=7, hidden_size=4, num_layers=1, batch_first=True))

In [ ]:
out, (ht, ct) = model3(x)
print(out.shape)
print(out)

torch.Size([3, 8, 4])
tensor([[[-0.3192, -0.1810, -0.2269,  0.1146],
         [ 0.0510, -0.0829, -0.0988,  0.0404],
         [-0.2568, -0.1813, -0.3272, -0.3018],
         [-0.0921, -0.0542, -0.2789,  0.0135],
         [ 0.1287,  0.0024, -0.1911, -0.0494],
         [ 0.2700,  0.0978,  0.2691, -0.2582],
         [ 0.2793,  0.1105,  0.0092, -0.2921],
         [ 0.5470,  0.1818,  0.5578, -0.5545]],

        [[-0.2865, -0.1173, -0.2475, -0.3089],
         [ 0.1918, -0.0478,  0.0890, -0.3259],
         [ 0.2239, -0.0286, -0.1275, -0.0715],
         [ 0.1095,  0.3398,  0.2685, -0.5337],
         [ 0.0090,  0.1245,  0.0043, -0.2754],
         [ 0.0213,  0.2292,  0.3392, -0.0887],
         [ 0.3603,  0.1635,  0.2238, -0.0830],
         [ 0.5180,  0.2332,  0.2149, -0.2801]],

        [[-0.1414, -0.0557, -0.1811,  0.1257],
         [ 0.1435, -0.0436, -0.0952, -0.1092],
         [-0.0116, -0.0990, -0.2736, -0.1505],
         [ 0.1577, -0.0423, -0.2243,  0.0445],
         [-0.1379, -0.1788, -0.427